In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# from sklearn.neighbors import NearestNeighbors
%matplotlib inline

In [2]:
df = pd.read_csv("ravdess_features.csv")

excluded = [
    "modality",
    "vocal_channel",
    "emotional_intensity",
    "statement",
    "repetition",
    "actor",
    "channels",
    "sample_width",
    "frame_rate",
    "frame_width",
    "sample_width",
    "stft_max",
    "frame_count",
    "length_ms",
    "mean",
    "max","min",
    # "kur","skew",
]
df.drop(columns=excluded, inplace=True)

Now missing intensity values are filled using $$I = 20 \; \log_{10}\left(\frac{\sigma}{\sigma_0}\right)$$

In [3]:
import intensity_fill
df = intensity_fill.fill_intensity(df)
df

/home/djanloo/Desktop/dynamiting/intensity_fill.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.intensity[df.intensity.isna()] = I_from_std(df["std"][df.intensity.isna()])


,emotion,sex,intensity,zero_crossings_sum,mfcc_mean,mfcc_std,mfcc_min,mfcc_max,sc_mean,sc_std,...,sc_kur,sc_skew,stft_mean,stft_std,stft_min,stft_kur,stft_skew,std,kur,skew
0,fearful,F,-36.793432,16995,-33.485947,134.654860,-755.22345,171.69092,5792.550744,3328.055457,...,-1.120769,0.250940,0.415250,0.335533,0.000000,-1.215025,0.403514,0.014482,9.406061,0.273153
1,angry,F,-33.739051,13906,-29.502108,130.485630,-713.98560,205.00770,5197.620555,4040.931570,...,-1.531204,0.219582,0.417488,0.352380,0.000000,-1.384174,0.290905,0.020417,19.564187,0.292157
2,happy,F,-32.290737,18723,-30.532463,126.577110,-726.06036,165.45653,4830.743037,3332.131300,...,-1.130153,0.436699,0.379758,0.352270,0.000000,-1.242947,0.470350,0.024317,4.881241,0.302659
3,surprised,F,-49.019839,11617,-36.059555,159.725160,-842.94635,190.03609,5376.446478,4053.663073,...,-1.497765,0.098802,0.407277,0.360552,0.000000,-1.445318,0.274756,0.003561,13.040259,-0.081014
4,happy,F,-31.214503,15137,-31.405996,122.125824,-700.70276,161.13400,5146.012474,3442.724109,...,-1.167544,0.496066,0.415974,0.350655,0.000000,-1.366742,0.318926,0.027522,7.485109,0.414872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,calm,M,-45.519766,9871,-30.225578,158.845500,-855.92420,208.18497,5886.647366,4114.260257,...,-1.588861,0.184390,0.573838,0.304496,0.000000,-1.075888,-0.310088,0.005260,9.551363,0.759247
2448,calm,M,-43.342901,8963,-31.160332,157.499700,-825.37410,220.79490,6602.742127,4191.727090,...,-1.614582,-0.113920,0.591010,0.306333,0.000000,-1.026071,-0.408018,0.006836,22.517991,1.073633
2449,sad,M,-36.197674,9765,-26.135280,138.133210,-768.36536,235.53938,4654.156076,4231.511986,...,-1.337419,0.619346,0.467994,0.347843,0.000000,-1.473789,0.130310,0.015384,4.964319,-0.005821
2450,surprised,M,-45.751265,9716,-28.242815,159.943400,-868.09576,219.84350,5524.736973,3596.031841,...,-1.465594,0.206481,0.578854,0.308684,0.001565,-1.277117,-0.242668,0.005182,13.855588,-0.110966


In [4]:
for field in ["sex", "emotion"]:
    df[field] = pd.Categorical(df[field]).codes


The first missing feature that we try to fill is the `actor` field. We follow a knn clustering strategy

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram

cols = [col for col in df.columns if col != 'sex']
X = df[cols].values

scaler = StandardScaler()
X_scal = scaler.fit_transform(X)
X_scal

scaler = MinMaxScaler()
X_minmax = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=2)
kmeans.fit(X_minmax)

KMeans(n_clusters=2)